# In this file we create an ANN model ( testing Purpose ) and fine tune this model for production and deployment.

In [2]:
##  Import Libraries

import pandas as pd
from keras.src.models import Sequential
from keras.src.layers import Dense , Dropout
from keras.src.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

In [3]:
## Spliting data into train and test data

df = pd.read_csv('../Data/Clean_Raw_Data')

X=df.iloc[:,:-1]
y=df['NObeyesdad']

x_train,  x_test, y_train , y_test =train_test_split(X,y, test_size=0.2, random_state=42)

x_train.shape,  x_test.shape, y_train.shape , y_test.shape

((1688, 16), (423, 16), (1688,), (423,))

In [4]:
## One-Hot encoding for target vaariable
y_test = to_categorical(y_test, 7)
y_train = to_categorical(y_train, 7)


In [5]:
## Scaling the data by using StandardScaler


scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Building the ANN model

In [6]:
## Building the ANN model



model=Sequential()

model.add(Dense(128, activation='relu', input_shape=(x_train_scaled.shape[1],)))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(16, activation='relu'))

## Output Layer.
model.add(Dense(7, activation='softmax'))


# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
## Train the model

history = model.fit(x_train_scaled, y_train, epochs=500, batch_size=32, validation_split=0.2)


Epoch 1/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.2884 - loss: 1.8700 - val_accuracy: 0.5089 - val_loss: 1.5119
Epoch 2/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4703 - loss: 1.4581 - val_accuracy: 0.6036 - val_loss: 0.9789
Epoch 3/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5676 - loss: 1.0430 - val_accuracy: 0.6923 - val_loss: 0.7500
Epoch 4/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6866 - loss: 0.8171 - val_accuracy: 0.7899 - val_loss: 0.5890
Epoch 5/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7379 - loss: 0.6895 - val_accuracy: 0.8432 - val_loss: 0.4885
Epoch 6/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7519 - loss: 0.6270 - val_accuracy: 0.9024 - val_loss: 0.4029
Epoch 7/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7911 - loss: 0.5330 - val_accuracy: 0.9172 - val_loss: 0.3399
Epoch 8/500
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8244 - loss: 0.4686 - val_accuracy: 0.9053 -

In [8]:
test_loss, test_accuracy = model.evaluate(x_test_scaled, y_test)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9361 - loss: 0.3745
Test Loss: 0.37793996930122375
Test Accuracy: 0.936170220375061


In [11]:
import numpy as np 


new_input=[0,25.196214,1.686306,104.572712,1,1,3.0,3.0,2,0,1.152736,0,0.319156,1.0,2,3]
new_input=np.array(new_input).reshape(1,-1)
new_input=scaler.transform(new_input)
input_arr=np.array(new_input).reshape(-1,16)


result=model.predict(input_arr)
result

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([[7.7676017e-13, 8.7808557e-20, 3.0934539e-09, 5.8093059e-07,
        9.9999940e-01, 2.9123020e-20, 1.3383398e-17]], dtype=float32)

In [17]:
max(result[0])

np.float32(0.9999994)

In [18]:
pred = np.argmax(result, axis=1)
int(pred)

4

# HyperPerimeter Tuning for the above model...